In [4]:
import os
 
class usbtmc:
    """Simple implementation of a USBTMC device driver, in the style of visa.h"""
 
    def __init__(self, device):
        self.device = device
        self.FILE = os.open(device, os.O_RDWR)
 
        # TODO: Test that the file opened
 
    def write(self, command: [bytes, str], response: bool=False):
        if isinstance(command, str):
            command = command.encode()
        os.write(self.FILE, command);
        if response:
            return self.read()
 
    def read(self, length = 1024):
        return os.read(self.FILE, length).decode()
 
    def name(self):
        self.write("*IDN?")
        return self.read(300)
 
    def reset(self):
        self.write("*RST")
 
    def __enter__(self):
        print("in __enter__")
        return self
    def __exit__(self, exception_type, exception_value, traceback):
        print("in __exit__")
 

from cached_property import cached_property
class RigolScope:
    """Class to control a Rigol DS1000 series oscilloscope"""
    def __init__(self, device):
        self.meas = usbtmc(device)
 
        self.name = self.meas.name()
 
    def write(self, command):
        """Send an arbitrary command directly to the scope"""
        self.meas.write(command)
 
    def read(self, command):
        """Read an arbitrary amount of data directly from the scope"""
        return self.meas.read(command)
 
    def reset(self):
        """Reset the instrument"""
        self.meas.reset()
        
    @cached_property
    def __idn__(self):
        return self.meas.name()
    
    @property
    def model(self):
        return self.__idn__.split(",")[1]
    
    @property
    def vendor(self):
        return self.__idn__.split(",")[0]
    
    @property
    def serial(self):
        return self.__idn__.split(",")[2]
    
    @property
    def version(self):
        return self.__idn__.split(",")[3]
    
    def __repr__(self):
        return f"{self.__class__.__name__}<{self.model}>"
    
    def __enter__(self):
        print("in __enter__")
        return self
    def __exit__(self, exception_type, exception_value, traceback):
        print("in __exit__")

r = RigolScope("/dev/usbtmc0")
r

RigolScope<DS1052D>

In [5]:
scope = r

In [6]:
assert scope.vendor=="Rigol Technologies"
assert scope.model=="DS1052D"
assert scope.version=="00.04.02.01.00"

In [7]:
r.meas.write(":MEASure:NWIDth? CHANNEL1".encode())

In [8]:
r.meas.read()

'5.00e-04'

In [9]:
r.meas.write(":MEASure:PDUTycycle? CHANNEL1".encode())
r.meas.read()

'5.00e-01'

In [59]:
class Channel:
    def __init__(self, number: [int], device: [usbtmc]):
        self.number = number
        self.device = device
        
    def __repr__(self):
        return f"{self.__class__.__name__}<{self.number}>"

    @property
    def source(self):
        return f"CHANNEL{self.number}"

In [60]:
c = Channel(1, r.meas)
c

Channel<1>

In [58]:
c.source()

'CHANNEL1'

In [ ]:
with open("../python_Rigol/DS1000DE_ProgrammingGuide_EN.txt", "r") as file:
    DS1000DE_ProgrammingGuide_EN = file.read()

In [36]:
command_re = re.compile(":([^?]+)\? \[<source")
for result in command_re.findall(DS1000DE_ProgrammingGuide_EN):
    print(result)


:MEASure:VPP

:MEASure:VMAX

:MEASure:VMIN

:MEASure:VAMPlitude

:MEASure:VTOP

:MEASure:VBASe

:MEASure:VAVerage

:MEASure:VRMS

:MEASure:OVERshoot

:MEASure:PREShoot

:MEASure:FREQuency

:MEASure:RISetime

:MEASure:FALLtime

:MEASure:PERiod

:MEASure:PWIDth

:MEASure:NWIDth

:MEASure:PDUTycycle

:MEASure:NDUTycycle

:MEASure:PDELay

:MEASure:NDELay

:WAVeform:DATA


In [44]:
command_re = re.compile(":([^?]+)\? \[<source")
for result in command_re.findall(DS1000DE_ProgrammingGuide_EN):
    query_command=result.strip()
    print()
    break

In [94]:
def measurement_factory(query_string:str, length:int=1024):
    query_string=str(query_string)
    query_string=query_string.strip()
    if not query_string.endswith("?"):
        query_string=f"{query_string}?"
    def get_measurement(self):
        self.device.write(f"{query_string} {self.source}")
        response = self.device.read(length=length)
        try:
            return float(response)
        except:
            return response
        
    return get_measurement

def measurement_factory2(query_string:str, length:int=1024):
    attr_name = query_string.strip("?").split(":")[-1].lower()
    return attr_name, measurement_factory(query_string, length)

In [95]:
class Measure():
    def __init__(self, channel):
        self.channel = channel
        self.device = self.channel.device
           

In [96]:
setattr(Channel, "vpp", measurement_factory(":MEASure:VPP?"))

c.vpp()

13.8

In [97]:
measurement_factory(":MEASure:VPP")

<function __main__.measurement_factory.<locals>.get_measurement(self)>

In [98]:
measurement_factory2(":MEASure:VPP")

('vpp', <function __main__.measurement_factory.<locals>.get_measurement(self)>)

In [81]:
attr_name, attr_get_function=Measure(c)

In [85]:
":MEASure:VPP".strip("?").split(":")[-1].lower()

'vpp'

In [102]:
for result in command_re.findall(DS1000DE_ProgrammingGuide_EN):
    result = result.strip()
    if result.startswith(":MEASure:"):
        attr_name, attr_get_function=measurement_factory2(result)
        setattr(Measure, attr_name, attr_get_function)

In [103]:
m

AttributeError: 'Measure' object has no attribute 'vpp'

In [76]:
setattr(Channel, "vpp", measurement_factory(":MEASure:VPP?"))


In [45]:
c.device.write("ac")
float(c.device.read())

13.6

In [46]:
result

':MEASure:VPP'

In [18]:
print(DS1000DE_ProgrammingGuide_EN)

RIGOL
Programming Guide

DS1000E, DS1000D
Series Digital Oscilloscope

Sept. 2010
RIGOL Technologies, Inc.

RIGOL

Copyright
© 2009 RIGOL Technologies, Inc. All Rights Reserved.

Trademark Information
RIGOL is registered trademark of RIGOL Technologies, Inc.

Notices







RIGOL products are protected by patent law in and outside of P.R.C..
RIGOL Technologies, Inc. reserves the right to modify or change parts of or all
the specifications and pricing policies at company’s sole decision.
Information in this publication replaces all previously corresponding material.
RIGOL shall not be liable for losses caused by either incidental or consequential
in connection with the furnishing, use or performance of this manual as well as
any information contained.
Any part of this document is forbidden to copy or photocopy or rearrange
without prior written approval of RIGOL.

Product Certification
RIGOL guarantees this product conforms to the standards of national and industrial.

Meanwhile,